In [1]:
import requests
import pandas as pd
import json
import time
import random

In [2]:
def request_url(base_url, api_key, q, begin_date, end_date, page):
    BASE_URL = base_url
    payload = {
        'api-key': api_key,
        'q': q,
        'begin_date': begin_date,
        'end_date': end_date,
        'page': page
        }
    resp = requests.get(url=BASE_URL, params=payload)
    assert resp.status_code == 200, "Request failed."
    data = resp.json()['response']
    return data

In [65]:
from datetime import datetime, timedelta

def split_date_range_half_month(begin_date, end_date):
    # 将字符串转换为 datetime 对象
    start = datetime.strptime(begin_date, "%Y%m%d")
    end = datetime.strptime(end_date, "%Y%m%d")

    # 保存每半月的查询范围
    date_ranges = []

    # 生成每半月起始和结束日期
    current = start
    while current <= end:
        # 当前月的第一天
        first_day_of_month = current.replace(day=1)
        # 当前月的最后一天
        next_month = (current.replace(day=28) + timedelta(days=4)).replace(day=1)
        last_day_of_month = next_month - timedelta(days=1)

        # 上半月
        mid_day = first_day_of_month.replace(day=15)
        if current <= mid_day:
            range_end = min(mid_day, end)
            date_ranges.append((current.strftime("%Y%m%d"), range_end.strftime("%Y%m%d")))

        # 下半月
        second_half_start = mid_day + timedelta(days=1)
        if second_half_start <= end:
            range_end = min(last_day_of_month, end)
            date_ranges.append((second_half_start.strftime("%Y%m%d"), range_end.strftime("%Y%m%d")))

        # 移动到下个月
        current = next_month

    return date_ranges

# 测试示例
begin_date = '20230616' #'20220224'
end_date = '20241115'
date_ranges = split_date_range_half_month(begin_date, end_date)

# for start, end in date_ranges:
#     print(f"Begin Date: {start}, End Date: {end}")

In [ ]:
api_key = 'WMbNP3qmAtA0mLDKInZuswXHXKNQSAA5' #K8J0AlmbcwIcUwkqTjt5UA80R70vLXH1 WMbNP3qmAtA0mLDKInZuswXHXKNQSAA5 Xm3gQbD5zBh75CBCYtGr18koPjCAiZGv iQSVR1fuEwqlW8yPDhvlLAGPXtwAhHq9
data = []
base_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json"
q = 'Ukraine'
for begin_date, end_date in date_ranges:
    print(f'正在获取{begin_date}-{end_date}期间主题为{q}的新闻检索第1页')
    page = 0
    dat = request_url(base_url, api_key, q, begin_date, end_date, page)
    hits = dat['meta']['hits']
    data += dat['docs']
    print(f'{begin_date}-{end_date}期间主题为{q}的新闻检索共{(hits + 9) // 10}页')
    print(f'{begin_date}-{end_date}期间主题为{q}的新闻检索第1页获取完成')
    time.sleep(20)
    for j in range(1, (hits + 9) // 10):
        print(f'正在获取{begin_date}-{end_date}期间主题为{q}的新闻检索第{j+1}页')
        page = j
        dat = request_url(base_url, api_key, q, begin_date, end_date, page)
        data += dat['docs']
        print(f'{begin_date}-{end_date}期间主题为{q}的新闻检索第{j+1}页获取完成')
        time.sleep(20)

In [63]:
df = pd.DataFrame(data)
df.to_csv('20230601-20230615期间主题为Ukraine的新闻检索.csv', index=False)

In [65]:
df[:2]

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name,print_section,print_page
0,Here is what we can learn about the Russian mi...,https://www.nytimes.com/2024/09/22/magazine/de...,Here is what we can learn about the Russian mi...,"Over the last year and a half, I reported from...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'The Heavy Toll of Desertion From the...,"[{'name': 'subject', 'value': 'Desertion (Mili...",2024-09-22T15:59:10+0000,article,Magazine,Magazine,"{'original': 'By Sarah A. Topol', 'person': [{...",News,nyt://article/55d24bbe-c81e-58e6-9b94-bbf5ee6a...,1382,nyt://article/55d24bbe-c81e-58e6-9b94-bbf5ee6a...,NaN,NaN,NaN
1,"At least 11,520 civilians had been killed in t...",https://www.nytimes.com/2024/09/03/world/europ...,"At least 11,520 civilians had been killed in t...",The Russian missile strike Tuesday on a milita...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'A Timeline of Some of the Deadliest ...,"[{'name': 'subject', 'value': 'Civilian Casual...",2024-09-03T21:25:27+0000,article,Foreign,World,"{'original': 'By Eve Sampson', 'person': [{'fi...",News,nyt://article/007ba0db-2193-5f5f-9183-b128bee1...,472,nyt://article/007ba0db-2193-5f5f-9183-b128bee1...,Europe,NaN,NaN
